# Comment Analysis


**Objective**: 

Get the most important topics using an LDA across hashtags by using an LDA analysis.     

**Contents**: <br>
**1. Data Import - Comments**<br>
**2. Preprocessing - Cleaning and Tokenization**<br>
**4. Data Transformation - Bigrams, Trigrams and Lemmatization**<br>
**5. Topic modeling using LDA**<br>
**6. Demo**<br>
**7. Clustering**

### Import libraries

In [ ]:
#pip install packages to perform topic analysis
!pip install gensim
!pip install pprint
!pip install pyldavis
!pip install pyLDAvis
!pip install spacy
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
#!pip install pandas==1.1.5

ERROR: Could not find a version that satisfies the requirement pprint
ERROR: No matching distribution found for pprint


  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019121 sha256=ad4bf0085bbd47e13fc42b666670a8182de8ab4f425c45be4776737283cb9724
  Stored in directory: c:\users\alfon\appdata\local\pip\cache\wheels\fc\31\e9\092e6f05b2817c9cb45804a3d1bf2b9bf6575742c01819337c
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.0.0
    Uninstalling en-core-web-sm-3.0.0:
      Successfully uninstalled en-core-web-sm-3.0.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 53.0.0
    Uninstalling setuptools-53.0.0:
      Successfully uninstalled setuptools-53.0.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.1
    Uninstalling thinc-8.0.1:
      Successfully uninstalled thinc-8.0.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.3
    Uninstalling spacy-3.0.3:
      Successfully uninstalled spacy-3.0.3


  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-04-04 19:23:46.693660: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2021-04-04 19:23:46.694839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#Import libraries for data pre-processing and LDA analysis
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('stopwords')
import gensim.corpora as corpora
import gensim as gensim

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alfon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alfon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Data Import - Comments

To perform our topic modelling we are using Instagram posts retrieved using the Instaloader API (code available in another notebook). For simplicty we import the data as csv, so we don't need to run the API several times.

In [ ]:
df = pd.read_csv('https://github.com/AlfonsoCabello/Social_Media_Analytics_042021/raw/main/instagram_loader_final.csv')
df.head()

,Username_ID,Caption,Comments,Comments_Text,Hashtags_Caption,Likes,Location,Mentions_Caption,Username
0,175509226,Happy Friday! I wore this colorful kimono romp...,27,"['The cutest romper 😍', 'You’re in my neck of ...","['liketkit', 'ltkunder50', 'ltktravel', 'ltkcu...",46,"PostLocation(id=216296668, name='Galveston, Te...","['pinklily', 'gapsmack87', 'shipleydonuts_galv...",pearlsandpigsblog
1,1599189434,#mrphotography #gainwithwestandmugweru #gainwi...,20,"['😍😍😍', '👏👏👏', '❤️', '❤️❤️❤️', '🔥🔥🔥', '🙌🙌', '❤...","['mrphotography', 'gainwithwestandmugweru', 'g...",20,"PostLocation(id=1760813094235018, name='Jhujha...",[],sameer.choudhary6375
2,2491624,@antoniopelayo x @isaacpelayo \n\nEdition of 5...,18,['#isaacpelayo #antoniopelayo #dope #love #art...,[],248,"PostLocation(id=212999109, name='Los Angeles, ...","['antoniopelayo', 'isaacpelayo']",isaacpelayo
3,47338304181,#digitalcollage #digitalartwork #digitalcollag...,8,"['🤩👌', '👏👏👏😍😍😍', 'Magnifique.👏👏👏👏👏', '🖤✨🤓✨🖤 Fa...","['digitalcollage', 'digitalartwork', 'digitalc...",90,NaN,[],thedi0nys0s
4,40846095404,#gainwithwestandmugweru #gainwithmchina#gainwi...,8,"['😘😘😘', '😍😍😍', '😍😍😍', '🔥🔥🔥', '😘😘😘', '❤️❤️❤️', ...","['gainwithwestandmugweru', 'gainwithmchina', '...",8,NaN,[],soyal_hamza


### 2. Preprocessing -  Cleaning and tokenization

In this step our goal is to prepare the data for the modeling and delete words or simbols that do not add value to the analysis.

In [ ]:
#Remove punctuations
df['Comments_Text'] = np.array(list(x.translate(str.maketrans("","", (string.punctuation+'•`'))) for x in list(df['Comments_Text'])))

In [ ]:
#Comments to lowecase
df['Comments_Text'] = np.array([x.lower() for x in list(df['Comments_Text'])])

In [ ]:
#Delete whitespaces
df['Comments_Text'] = np.array([x.strip() for x in list(df['Comments_Text'])])

In [ ]:
#Tokenize words
df['Comments_Text'] = np.array([word_tokenize(x) for x in list(df['Comments_Text'])])

In [ ]:
#Remove stopwords
sw = set(stopwords.words('english'))
df['Comments_Text'] = np.array([[i for i in x if i not in sw] for x in list(df['Comments_Text'])])

In [ ]:
#Replace numbers to words
array = np.array([['0', 'zero'], ['1', 'one'], ['2', 'two'], ['3', 'three'], ['4', 'four'], ['5', 'five'], ['6', 'six'], ['7', 'seven'], ['8', 'eight'], ['9', 'nine'], ['10', 'ten'], ['12', 'twelve'], ['20', 'twenty']])
nums = pd.DataFrame(array, columns = ['Number', 'Word'])

#Loop each token, if it's a number change for a word
for x in list(df['Comments_Text']):
    for idx, number in enumerate(x):
        if number in list(nums['Number']):
            x[idx] = nums[nums['Number'] == number]['Word'].to_list()[0]

In [ ]:
#Remove websites, hashtags and years
for toke in list(df['Comments_Text']):
    for idx, i in enumerate(toke):
        toke[idx] = re.sub(r'\d+', '', i)
        toke[idx] = re.sub(r'www', '', i)
        toke[idx] = re.sub(r'#', '', i)
        toke[idx] = re.sub(r'com', '', i)
    toke = list(filter(lambda x: x != '', toke))

In [ ]:
#Comments to be analized
#We analyze posts that have comments (at least 1)

data_words = df[df['Comments_Text'].map(lambda d: len(d)) > 0]
data_words = data_words['Comments_Text']
data_words

0      [cutest, romper, 😍, ’, neck, woods, romper, pe...
1      [😍😍😍, 👏👏👏, ❤️, ❤️❤️❤️, 🔥🔥🔥, 🙌🙌, ❤️❤️, 🙌, 🔥🔥🔥, ...
2      [isaacpelayo, antoniopelayo, dope, love, art, ...
3      [🤩👌, 👏👏👏😍😍😍, magnifique👏👏👏👏👏, 🖤✨🤓✨🖤, fantástic...
4      [😘😘😘, 😍😍😍, 😍😍😍, 🔥🔥🔥, 😘😘😘, ❤️❤️❤️, ❤️❤️❤️, 🔥🔥🔥,...
                             ...                        
219                                         [❤️❤️❤️❤️❤️]
220                                     [art, 🤍🤍🤍🤍🤍🤍🤍🤍💙]
221                                  [brilliant, ❤️❤️❤️]
222    [unfortunately, two, photos, didnt, load, next...
223                                         [👏👏👏💚🍀💚🍀💚🍀🤍]
Name: Comments_Text, Length: 223, dtype: object

### 3. Data Transformation

To fit the data into the LDA model we need to take some preprocessing steps such as bigrams and lemmatization that significantly improves the performance of the topic modelling.

In [ ]:
#Next we create a dictionary with the text to be analized
import gensim.corpora as corpora
import gensim as gensim

id2word = corpora.Dictionary(data_words)
texts = data_words

#Determine frequency of the words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 2), (9, 2), (10, 1), (11, 2), (12, 3), (13, 1), (14, 2), (15, 1), (16, 5), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1)]


Bigrams are two words frequently occurring together in the comment. Trigrams are 3 words frequently occurring.

In [ ]:
#Define bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=200) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=200)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
#Functions that create the bigrams abd trigams and loop through each word in the text to be analized 
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
##Important to import the en_core_web_sm##
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

#Fit bigrams
data_words_bigrams = make_bigrams(data_words)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

#lemmatize dataset that has been processed with bigrams
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['cut', 'romper', '😍', 'neck', 'wood', 'romper', 'perfect', 'beach', 'vacation', 'cut', 'romper', 'perfect', 'tropical', 'vacay', 'romper', 'cute', 'love', '❤', '️', '❤', '️', '❤', '️', '❤', '️', '❤', '️', '❤', '️', 'affordablefashion', 'momstyle', 'outfitinspiration', 'outfitidea', 'onlineshopping', 'styleblogg', 'stylish', 'ootdinspo', 'outfitdetail', 'effortlessstyle', 'galvestontx', 'beachstyle', 'vacationstyle', 'beachvacation', 'muralart', 'pinklilyboutique', 'romperseason', 'romperstyle', 'springstyle', 'lifestyleblogger', 'cute', '😍', 'thank', 'love', 'love', 'fun', 'print', '❤', '️', 'love', 'mural', 'background', 'fun', 'romper', 'cute', 'fun', 'wall', 'happy', 'friday', 'love', 'romper', 'happy', 'friday', 'well', 'pretty', 'sweet', '✨', 'dm', 'aphroditetreasure', 'sweetie', 'crescendojewel', '💙']]


In [ ]:
import gensim.corpora as corpora

#Next we create a new dictionary with the transformed text
#These are the main inputs for the LDA
id2word = corpora.Dictionary(data_lemmatized)
#Create Corpus
texts = data_lemmatized
#Term Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 3), (9, 1), (10, 1), (11, 2), (12, 3), (13, 1), (14, 2), (15, 1), (16, 5), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 6), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 7), (48, 7), (49, 1), (50, 2)]]


Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (16, 5) above implies, word id 16 occurs five times in the first comment.

### 4. Topic Modelling Analysis

Initially we are creating a model with 3 topics to measure how coherent are our topics, then we are doing a grid search to find the optimal topics.

#### 4.1. Initial Model

In [ ]:
#Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=3, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint

# Print the keywords in the 10 topics and its weights
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.038*"😍" + 0.018*"👏" + 0.015*"promote" + 0.009*"virtuosoig" + 0.007*"art" '
  '+ 0.006*"muralart" + 0.006*"mural" + 0.005*"dm" + 0.005*"streetart" + '
  '0.005*"👌"'),
 (1,
  '0.044*"🔥" + 0.016*"art" + 0.013*"mural" + 0.012*"love" + 0.007*"muralart" + '
  '0.007*"urbanart" + 0.006*"😍" + 0.006*"artist" + 0.006*"artwork" + '
  '0.006*"paint"'),
 (2,
  '0.090*"❤" + 0.082*"️" + 0.030*"🔥" + 0.011*"dm" + 0.009*"😍" + 0.008*"love" + '
  '0.006*"amazing" + 0.006*"romper" + 0.005*"virtuosoig" + 0.005*"mural"')]


In [ ]:
from gensim.models import CoherenceModel

#Get Coherance score

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.49117982022828954


From the previous analysis we got a cohereance score of .49 which means that are our model is achievieng to represent the main topics but still needs fune tunning to perform better.

####  4.2. Grid Search

In [ ]:
#Model to be used in the grid search to find best parameters 
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
#Grid serach
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
100%|██████████| 540/540 [12:41<00:00,  1.41s/it]


In [ ]:
results = pd.DataFrame(model_results)
results.to_csv('results.csv')

Best parameters:
Topics = 3 
Alpha = 0.01
Beta = 0.31

#### 4.3.  Final Model

In [ ]:
#Final optimized model

k = 3
alpha= .01
beta = .31

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=beta)

In [ ]:
#Get Coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5009183892327541


#### 4.4. Words in Topics

From the model above we have come up with 5 topics that describe the comments gathered. Next we are analizing each topic and comparing its coherence score to determine which topics are semantically interpretable.

In [ ]:
from pprint import pprint

# Print the keywords in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"😍" + 0.019*"👏" + 0.008*"mural" + 0.008*"️" + 0.007*"muralart" + '
  '0.006*"streetart" + 0.005*"promote" + 0.005*"👌" + 0.005*"que" + 0.005*"❤"'),
 (1,
  '0.042*"🔥" + 0.019*"art" + 0.013*"love" + 0.011*"😍" + 0.010*"mural" + '
  '0.010*"virtuosoig" + 0.010*"promote" + 0.009*"❤" + 0.006*"urbanart" + '
  '0.006*"muralart"'),
 (2,
  '0.085*"❤" + 0.079*"️" + 0.030*"🔥" + 0.011*"dm" + 0.009*"😍" + 0.009*"love" + '
  '0.006*"mural" + 0.006*"romper" + 0.005*"amazing" + 0.005*"art"')]


In [ ]:
#Get each topic and extract the words into a dataframe
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])
    
finalData = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'CoheranceScore'])

#### 4.5. Topics Importance

From the analysis, we got that the most coherent topic is topic #3.

In [ ]:
#Get each topic importance
finalData_topics = pd.DataFrame(finalData.groupby('Topic')['CoheranceScore'].mean())

#Determine the topic with the highest score
highSC = pd.DataFrame(finalData_topics.loc[finalData_topics['CoheranceScore'].idxmax()]).columns[0]
finalData_topics


,CoheranceScore
Topic,
0,0.010331
1,0.013610
2,0.024468


#### 4.6. Most Important Words in Topic 3

In [ ]:
#Get words
finalData_words = finalData[(finalData.Topic==highSC)]
finalData_words

,Topic,Word,CoheranceScore
20,2,❤,0.085328
21,2,️,0.078904
22,2,🔥,0.030348
23,2,dm,0.010654
24,2,😍,0.009091
25,2,love,0.008549
26,2,mural,0.005804
27,2,romper,0.005799
28,2,amazing,0.005315
29,2,art,0.004893


In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.016168 -0.106108       1        1  36.043798
2     -0.107293  0.040084       2        1  33.962417
0      0.091125  0.066023       3        1  29.993785, topic_info=        Term       Freq      Total Category  logprob  loglift
47         ❤  71.000000  71.000000  Default  30.0000  30.0000
48         ️  65.000000  65.000000  Default  29.0000  29.0000
51         🔥  57.000000  57.000000  Default  28.0000  28.0000
85         👏  15.000000  15.000000  Default  27.0000  27.0000
50         😍  36.000000  36.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
47         ❤   3.232173  71.483928   Topic3  -5.2823  -1.8921
16      love   2.209123  18.521688   Topic3  -5.6629  -0.9222
72     paint   1.466896   7.217326   Topic3  -6.0723  -0.3892
55      arte   1.455851   4.541130   Topic3  -6.0799   0.0666
663  wallart   1.452596   3.198880   Topic3  -6.0821   0.4147

[185 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
302       1  0.507583    abstract
302       2  0.507583    abstract
284       1  1.025225  aerosolart
238       2  0.516566          ai
175       3  0.536274       algun
...     ...       ...         ...
229       1  0.762826           😚
52        1  0.254288           😮
52        2  0.762864           😮
565       1  0.762769    🤍🤍🤍🤍🤍🤍🤍🤍
274       2  1.033688           🦓

[191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])